# Aquisição e Limpeza de dados

Esse notebook documenta o processamento dos datasets baixados do INMET, com os agrupamentos necessários pra criar os arquivos `dados.csv`, `completo.csv` e `metadados.csv`.

Baixe o arquivo do site [Dados históricos do INMET](https://portal.inmet.gov.br/dadoshistoricos), selecionando o ano de 2019 todo.

Descompacte em uma pasta, seguindo a estrutura:
``` bash
- baixado
--- descompactado
```

Estamos prontos para ver os arquivos, mas mesmo sem abrir eles podemos ter alguma informação sobre esses dados.

Vamos listar todos os arquivos e ver quantas cidades tem por estado.

In [1]:
import os
import pandas as pd
import numpy as np
import altair as alt
import json
from io import StringIO
import unicodedata
import re

In [2]:
prefixo = "../baixado/descompactado/"
nome_arquivos = os.listdir(prefixo)
print(nome_arquivos[:4])

['INMET_CO_DF_A001_BRASILIA_01-01-2019_A_31-12-2019.CSV', 'INMET_CO_DF_A042_BRAZLANDIA_01-01-2019_A_31-12-2019.CSV', 'INMET_CO_DF_A045_AGUAS EMENDADAS_01-01-2019_A_31-12-2019.CSV', 'INMET_CO_DF_A046_GAMA (PONTE ALTA)_01-01-2019_A_31-12-2019.CSV']


In [3]:
siglas_estado = [nome.split("_") for nome in nome_arquivos]
print(siglas_estado[:2]) # tem mais informações que só as iniciais
siglas_estado = [nome[2] for nome in siglas_estado]
print(siglas_estado[:20]) # agora está certo
siglas = list(set(siglas_estado))
siglas.sort()
print(siglas) # todos os estados estão presentes?

[['INMET', 'CO', 'DF', 'A001', 'BRASILIA', '01-01-2019', 'A', '31-12-2019.CSV'], ['INMET', 'CO', 'DF', 'A042', 'BRAZLANDIA', '01-01-2019', 'A', '31-12-2019.CSV']]
['DF', 'DF', 'DF', 'DF', 'DF', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO']
['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']


In [4]:
dados_estado = []
for s in siglas:
    dados_estado.append({"estado":s, "cidades": siglas_estado.count(s)})
dataframe = pd.DataFrame(dados_estado)
dataframe

,estado,cidades
0,AC,7
1,AL,7
2,AM,19
3,AP,4
4,BA,47
5,CE,16
6,DF,5
7,ES,13
8,GO,26
9,MA,17


A tabela não ajuda muito a ver a distribuição, é melhor com um gráfico

In [5]:
alt.Chart(dataframe).mark_bar().encode(
    x="estado",
    y="cidades"
)

alt.Chart(...)

Dá pra ver melhor a disparidade, mas no nosso caso queremos só o Pará.  
Vamos ler um arquivo com a sigla `PA` e ver como está o formato.

In [6]:
from pprint import pprint
with open(prefixo + nome_arquivos[0], "r") as arquivo:
    pprint(arquivo.readlines()[:20])

['REGI?O:;CO\n',
 'UF:;DF\n',
 'ESTAC?O:;BRASILIA\n',
 'CODIGO (WMO):;A001\n',
 'LATITUDE:;-15,789343\n',
 'LONGITUDE:;-47,925756\n',
 'ALTITUDE:;1160,96\n',
 'DATA DE FUNDAC?O:;07/05/00\n',
 'Data;Hora UTC;PRECIPITAÇÃO TOTAL, HORÁRIO (mm);PRESSAO ATMOSFERICA AO NIVEL '
 'DA ESTACAO, HORARIA (mB);PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) '
 '(mB);PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB);RADIACAO GLOBAL '
 '(W/m²);TEMPERATURA DO AR - BULBO SECO, HORARIA (°C);TEMPERATURA DO PONTO DE '
 'ORVALHO (°C);TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C);TEMPERATURA MÍNIMA '
 'NA HORA ANT. (AUT) (°C);TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) '
 '(°C);TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C);UMIDADE REL. MAX. NA '
 'HORA ANT. (AUT) (%);UMIDADE REL. MIN. NA HORA ANT. (AUT) (%);UMIDADE '
 'RELATIVA DO AR, HORARIA (%);VENTO, DIREÇÃO HORARIA (gr) (° (gr));VENTO, '
 'RAJADA MAXIMA (m/s);VENTO, VELOCIDADE HORARIA (m/s);\n',
 '2019/01/01;0000 '
 'UTC;1;887;887;886,6;;18,5;17;18,7;18,4;17,3

Tem muita informação nas primeiras linhas do que a gente precisa, então vamos dividir: um arquivo de metadados e um arquivo com a informação dos sensores. Vamos começar adicionando os cabeçalhos.

In [7]:
dados = "Cidade,"
metadados = ""

with open(prefixo + nome_arquivos[0], "r", encoding="latin-1") as arquivo:
    linhas = arquivo.readlines()
    dados += linhas[8][:-2].replace(","," |").replace(";",",") + "\n"
    metadados = ",".join([linha.split(":;")[0].replace("?","Ã").replace("CÃ","ÇÃ") for linha in linhas[:8]]) + "\n"
    print(metadados)
    print(dados)

REGIÃO,UF,ESTAÇÃO,CODIGO (WMO),LATITUDE,LONGITUDE,ALTITUDE,DATA DE FUNDAÇÃO

Cidade,Data,Hora UTC,PRECIPITAÇÃO TOTAL | HORÁRIO (mm),PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO | HORARIA (mB),PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (W/m²),TEMPERATURA DO AR - BULBO SECO | HORARIA (°C),TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),UMIDADE RELATIVA DO AR | HORARIA (%),VENTO | DIREÇÃO HORARIA (gr) (° (gr)),VENTO | RAJADA MAXIMA (m/s),VENTO | VELOCIDADE HORARIA (m/s)



O cabeçalho dos dados ficou com muita informação extra, e um formato não tão agradável. Trocar o nome das colunas é uma função comum em vários processos de visualização, e será feito manualmente agora. Mas vamos salvar uma estrutura de chave valor, já que em vários títulos também tem informação sobre as medidas.

In [8]:
dados1 = "Cidade,Data,Hora,Precipitação,Pressão Atmosférica ao nível da estação,\
Pressão Atmosférica máxima,Pressão Atmosférica mínima,Radiação Global,Temperatura do ar - bulbo seco,Temperatura do ponto de orvalho,\
Temperatura máxima,Temperatura mínima,Temperatura orvalho máxima,Temperatura orvalho mínima,\
Umidade Relativa máxima,Umidade Relativa mínima,Umidade Relativa do Ar,Direção Horária do Vento,Rajada Máxima de Vento,Velocidade Horária do Vento"

cabecalho = {}
for d,d1 in zip(dados.split(","), dados1.split(",")):
    cabecalho[d1] = d

print(cabecalho)

with open("../datasets/cabecalho.json", "w+", encoding="utf-8") as arquivo:
    arquivo.write(json.dumps(cabecalho))

dados = dados1 + "\n"

{'Cidade': 'Cidade', 'Data': 'Data', 'Hora': 'Hora UTC', 'Precipitação': 'PRECIPITAÇÃO TOTAL | HORÁRIO (mm)', 'Pressão Atmosférica ao nível da estação': 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO | HORARIA (mB)', 'Pressão Atmosférica máxima': 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)', 'Pressão Atmosférica mínima': 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)', 'Radiação Global': 'RADIACAO GLOBAL (W/m²)', 'Temperatura do ar - bulbo seco': 'TEMPERATURA DO AR - BULBO SECO | HORARIA (°C)', 'Temperatura do ponto de orvalho': 'TEMPERATURA DO PONTO DE ORVALHO (°C)', 'Temperatura máxima': 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 'Temperatura mínima': 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 'Temperatura orvalho máxima': 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)', 'Temperatura orvalho mínima': 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)', 'Umidade Relativa máxima': 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)', 'Umidade Relativa mínima': 'UMIDADE REL. MIN. NA HOR

Vamos adicionar os dados em dois arquivos, dos sensores e dos metadados.

In [9]:
def extrair_metadados(linhas):
    return ",".join([linha.split(":;")[1].replace("\n","").replace(",",".") for linha in linhas[:8]]) + "\n"

for caminho, sigla in zip([prefixo + n for n in nome_arquivos], siglas_estado):
    if sigla == "PA":
        with open(caminho, "r", encoding="latin-1") as arquivo:
            linhas = arquivo.readlines()
            metadados += extrair_metadados(linhas)
            linhas_dados = [caminho.split("_")[4] + "," + linha.replace(",",".").replace(";",",")[:-2] + "\n" for linha in linhas[9:]]
            dados += "".join(linhas_dados)

O nome das cidades não está condizente com o que vamos usar nos mapas, então vamos trocar pelos nomes no arquivo que será utilizado nos mapas.

In [10]:
# do link: https://stackoverflow.com/a/31607735
def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def text_to_id(text):
    """
    Convert input text to id.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    text = strip_accents(text)
    text = re.sub('[ ]+', '_', text)
#     text = re.sub('[^0-9a-zA-Z_-]', '', text)
    return text

df = pd.read_csv(StringIO(dados))
df_meta = pd.read_csv(StringIO(metadados))

print(df_meta.columns)

with open("../datasets/para_geo.json","r",encoding="utf-8") as f:
    geo = json.load(f)
    cidades = [g["properties"]["name"] for g in geo["features"]]
    print(cidades)
    for cidade in cidades:
        indexes_cidade = df["Cidade"] == text_to_id(cidade.upper()).replace("_", " ")
        indexes_df_meta = df_meta["ESTAÇÃO"] == text_to_id(cidade.upper()).replace("_", " ")
        if np.sum(indexes_cidade) > 0:
            df["Cidade"][indexes_cidade] = cidade
            df_meta["ESTAÇÃO"][indexes_df_meta] = cidade
    print(df["Cidade"].unique())
    print(df_meta["ESTAÇÃO"].unique())

Index(['REGIÃO', 'UF', 'ESTAÇÃO', 'CODIGO (WMO)', 'LATITUDE', 'LONGITUDE',
       'ALTITUDE', 'DATA DE FUNDAÇÃO'],
      dtype='object')
['Abaetetuba', 'Abel Figueiredo', 'Acará', 'Afuá', 'Água Azul do Norte', 'Alenquer', 'Almeirim', 'Altamira', 'Anajás', 'Ananindeua', 'Anapu', 'Augusto Corrêa', 'Aurora do Pará', 'Aveiro', 'Bagre', 'Baião', 'Bannach', 'Barcarena', 'Belém', 'Belterra', 'Benevides', 'Bom Jesus do Tocantins', 'Bonito', 'Bragança', 'Brasil Novo', 'Brejo Grande do Araguaia', 'Breu Branco', 'Breves', 'Bujaru', 'Cachoeira do Piriá', 'Cachoeira do Arari', 'Cametá', 'Canaã dos Carajás', 'Capanema', 'Capitão Poço', 'Castanhal', 'Chaves', 'Colares', 'Conceição do Araguaia', 'Concórdia do Pará', 'Cumaru do Norte', 'Curionópolis', 'Curralinho', 'Curuá', 'Curuçá', 'Dom Eliseu', 'Eldorado dos Carajás', 'Faro', 'Floresta do Araguaia', 'Garrafão do Norte', 'Goianésia do Pará', 'Gurupá', 'Igarapé-Açu', 'Igarapé-Miri', 'Inhangapi', 'Ipixuna do Pará', 'Irituia', 'Itaituba', 'Itupiranga', 

<ipython-input-10-b5ec39e11bd6>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cidade"][indexes_cidade] = cidade
<ipython-input-10-b5ec39e11bd6>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta["ESTAÇÃO"][indexes_df_meta] = cidade


['Belém' 'Castanhal' 'Medicilândia' 'Pacajá' 'Placas' 'Paragominas'
 'TOME ACU' 'Rondon do Pará' 'Salinópolis' 'São Félix do Xingu' 'Bragança'
 'Soure' 'Breves' 'Tucuruí' 'SERRA DOS CARAJAS' 'Itaituba' 'Óbidos'
 'Santana do Araguaia' 'Tucumã' 'Novo Repartimento' 'Cametá'
 'Monte Alegre' 'Marabá' 'Conceição do Araguaia' 'MINA DO PALITO'
 'Xinguara' 'Capitão Poço' 'Santarém' 'Dom Eliseu' 'Altamira' 'Redenção']
['Belém' 'Castanhal' 'Medicilândia' 'Pacajá' 'Placas' 'Paragominas'
 'TOME ACU' 'Rondon do Pará' 'Salinópolis' 'São Félix do Xingu' 'Bragança'
 'Soure' 'Breves' 'Tucuruí' 'SERRA DOS CARAJAS' 'Itaituba' 'Óbidos'
 'Santana do Araguaia' 'Tucumã' 'Novo Repartimento' 'Cametá'
 'Monte Alegre' 'Marabá' 'Conceição do Araguaia' 'MINA DO PALITO'
 'Xinguara' 'Capitão Poço' 'Santarém' 'Dom Eliseu' 'Altamira' 'Redenção']


Dá pra ver que no meio das cidades tem uma serra e uma mina, que não tem uma cidade referenciada mas tem sensores. Pra fins didáticos, vamos ignorar isso nas aulas. Só precisamos fazer uma troca manual, a cidade de Tomé-Açu.

In [11]:
df["Cidade"][df["Cidade"] == "TOME ACU"] = 'Tomé-Açu'
df_meta["ESTAÇÃO"][df_meta["ESTAÇÃO"] == "TOME ACU"] = 'Tomé-Açu'

<ipython-input-11-d9d7fb24a15e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Cidade"][df["Cidade"] == "TOME ACU"] = 'Tomé-Açu'
<ipython-input-11-d9d7fb24a15e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta["ESTAÇÃO"][df_meta["ESTAÇÃO"] == "TOME ACU"] = 'Tomé-Açu'


In [12]:
df["Cidade"].unique()

array(['Belém', 'Castanhal', 'Medicilândia', 'Pacajá', 'Placas',
       'Paragominas', 'Tomé-Açu', 'Rondon do Pará', 'Salinópolis',
       'São Félix do Xingu', 'Bragança', 'Soure', 'Breves', 'Tucuruí',
       'SERRA DOS CARAJAS', 'Itaituba', 'Óbidos', 'Santana do Araguaia',
       'Tucumã', 'Novo Repartimento', 'Cametá', 'Monte Alegre', 'Marabá',
       'Conceição do Araguaia', 'MINA DO PALITO', 'Xinguara',
       'Capitão Poço', 'Santarém', 'Dom Eliseu', 'Altamira', 'Redenção'],
      dtype=object)

In [13]:
df_meta.head()

,REGIÃO,UF,ESTAÇÃO,CODIGO (WMO),LATITUDE,LONGITUDE,ALTITUDE,DATA DE FUNDAÇÃO
0,N,PA,Belém,A201,-1.411228,-48.439512,21.17,20/01/03
1,N,PA,Castanhal,A202,-1.300875,-47.947967,47.13,24/01/03
2,N,PA,Medicilândia,A209,-3.510943,-52.963450,251.68,17/02/08
3,N,PA,Pacajá,A210,-3.843611,-50.638056,89.02,27/02/08
4,N,PA,Placas,A211,-3.864040,-54.216416,100.45,14/02/08


Repetimos o processo para os metadados, e vamos salvar tudo em disco. Vamos fazer uma transformação e juntar data e hora na mesma coluna.

In [14]:
# Juntando as colunas de data e hora
df["Data"] = pd.to_datetime(df['Data'] + ' ' + df['Hora'])
df = df.drop(["Hora"], axis=1)

# Salvando os dados e metadados
df.to_csv("../datasets/completo.csv", index=False)
df_meta.to_csv("../datasets/metadados.csv", index=False)

Por fim vamos agrupar pela média de medições pelo dia para criar um arquivo mais leve.

In [15]:
df_leve = df.groupby(["Cidade",pd.Grouper(key='Data', freq='W')]).mean()
df_leve.reset_index(level=df_leve.index.names, inplace=True)

df_leve = df_leve[df_leve["Cidade"].isin(df_leve["Cidade"].unique()[:20:2])]
df_leve["Cidade"].unique()
df_leve.to_csv("../datasets/dados.csv", index=False)

Deixamos tudo pronto para outra seção.